# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 
# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("../Weather/cities.csv")
data

,Unnamed: 0,City,lat,lon,temp,temp_max,humidity,cloudiness,wind_speed,country
0,0,Torbay,47.67,-52.73,29.46,30.99,92.0,90.0,8.05,CA
1,1,Port Alfred,-33.59,26.89,58.03,58.03,72.0,13.0,14.56,ZA
2,2,Hilo,19.73,-155.09,70.30,73.99,78.0,90.0,5.82,US
3,3,Luancheng,37.88,114.65,43.45,43.45,50.0,100.0,4.09,CN
4,4,Vanimo,-2.67,141.30,82.56,82.56,74.0,92.0,3.65,PG
...,...,...,...,...,...,...,...,...,...,...
572,640,Bayangol,50.70,103.47,7.11,7.11,71.0,0.0,2.71,RU
573,641,Port Hardy,50.70,-127.42,42.80,42.80,93.0,90.0,5.66,CA
574,642,Ponta Delgada,37.73,-25.67,51.80,51.80,71.0,75.0,8.05,PT
575,644,Saint-François,46.42,3.91,33.80,33.80,74.0,78.0,2.68,FR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = data[["lat", "lon"]].astype(float)
humidity = data[["humidity"]]

humidity = pd.to_numeric(humidity["humidity"], downcast = "float")

In [4]:
fig = gmaps.figure()

heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_data = data.loc[(data["temp_max"]>70) & (data["temp_max"] < 80) & (data["wind_speed"] < 10) & (data["cloudiness"] == 0)]
ideal_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 295 to 527
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  7 non-null      int64  
 1   City        7 non-null      object 
 2   lat         7 non-null      float64
 3   lon         7 non-null      float64
 4   temp        7 non-null      float64
 5   temp_max    7 non-null      float64
 6   humidity    7 non-null      float64
 7   cloudiness  7 non-null      float64
 8   wind_speed  7 non-null      float64
 9   country     7 non-null      object 
dtypes: float64(7), int64(1), object(2)
memory usage: 616.0+ bytes


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_data

hotel_df["Hotel_Name"] = ""
hotel_df



params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row["lat"]
    lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params)
    

    hotels = hotel.json()

    try:
        hotel_df.loc[index, "Hotel_Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

/Users/hunterblock/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/hunterblock/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel Union de Reyes
Portside Hotel - Heritage Collection
Hotel Provincial
Missing field/result... skipping.
Hotel Diya Disha
Salalah Gardens Hotel
Missing field/result... skipping.


,Unnamed: 0,City,lat,lon,temp,temp_max,humidity,cloudiness,wind_speed,country,Hotel_Name
295,327,Alacranes,22.77,-81.57,75.27,75.27,66.0,0.0,5.93,CU,Hotel Union de Reyes
308,340,Gisborne,-38.65,178.00,71.49,71.49,43.0,0.0,5.93,NZ,Portside Hotel - Heritage Collection
355,398,San Juan,-31.54,-68.54,73.62,73.62,35.0,0.0,8.03,AR,Hotel Provincial
363,408,Sompeta,18.93,84.60,79.36,79.36,56.0,0.0,5.06,IN,
368,413,Barpeta,26.32,91.00,71.35,71.35,53.0,0.0,2.77,IN,Hotel Diya Disha
425,475,Salalah,17.02,54.09,73.40,73.40,69.0,0.0,3.36,OM,Salalah Gardens Hotel
527,590,Adré,13.47,22.20,78.57,78.57,14.0,0.0,5.50,TD,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["lat", "lon"]]
info_box_template

'\n<dl>\n<dt>Name</dt><dd>{Hotel_Name}</dd>\n<dt>City</dt><dd>{City}</dd>\n<dt>Country</dt><dd>{country}</dd>\n</dl>\n'

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations ,info_box_content=[info_box_template])
fig.add_layer(markers)

# Display Map
fig

IndexError: list index out of range